In [1]:
import numpy as np
import pandas as pd

In [2]:
TR = "流水数据/table_loss_transaction_trainingd3/ata.txt"
TR_ = "流水数据/table_loss_transaction_testdata.txt"
TP = "抛出点数据/table_throwpoint_training.txt"
TP_ = "抛出点数据/table_throwpoint_test.txt" 

M = "商户数据/table_datamining_merchant.txt"
LBL = "特征值/lable.csv"
TRSC = "特征值/trsc_cut.csv"
EIGEN = "特征值/eigen_train.csv"
# EIGEN = "特征值/eigen_test.csv"

In [3]:
def sample(path):
    dt = pd.read_csv( path, engine='python', sep=",")
    samp = dt.sample(frac=0.01)
    samp.to_csv(path+"sample")
    return samp

In [4]:
'''trsc: 读入drop后的transaction数据 '''
# try:
#     trsc
# except NameError:
#     trsc_exists = False
#     trsc=pd.read_csv(TRSC,engine='python')
# else:
#     trsc_exists = True
# trsc
trsc=pd.read_csv(TRSC,engine='python')

In [5]:
 trsc.drop(columns=['pay_way','sub_pay_way','ctime'],inplace=True)

## 特征6： 交易类型、订单状态（成功与否）
**交易状态statue**：

创建 0-----> 0

成功 2000---> 1

其他 2001---> 0

**交易类型type**：

付款 30 ----> 1

取消 10 ----> 0

退款 11 ----> 0

In [7]:
trsc.drop(columns=['Unnamed: 0'],inplace=True)
trsc['status'].replace(2000,1,inplace=True)
trsc['status'].replace(2001,0,inplace=True)
trsc['type'].replace(30,1,inplace=True)
trsc['type'].replace(10,0,inplace=True)
trsc['type'].replace(11,0,inplace=True)

## 特征2：（支付通道中）是否优惠

price_reduced = bool(#original_amount - #effective_amount)

price_reduced = 1 : 有优惠

price_reduced = 0 : 无优惠

In [8]:
trsc['price_reduced']= trsc['original_amount']-trsc['effective_amount']

In [9]:
#确认没有负值
for i in trsc.price_reduced:
    if i<0:
        print("@")

In [10]:
trsc['if_price_reduced'] = trsc['price_reduced'].astype(bool).astype(int)

In [11]:
 sum(trsc.if_price_reduced)

870

## 特征3： 提供的支付方式个数

（4*1vector) 

#payw_offer = sum[bool(bankcard_debit), bool(wallet_weixin), bool(wallet_alipay), bool(alipay_huabei)]


In [12]:
store_mean = trsc.groupby(['store_id']).mean().reset_index()
store_mean['bankcard_debit'] = store_mean['bankcard_debit'].astype(bool).astype(int)
store_mean['wallet_alipay'] = store_mean['wallet_alipay'].astype(bool).astype(int)
store_mean['wallet_weixin'] =store_mean['wallet_weixin'].astype(bool).astype(int)
store_mean['alipay_huabei'] = store_mean['alipay_huabei'].astype(bool).astype(int)
store_mean['payw_offer']=store_mean['bankcard_debit']+store_mean['wallet_alipay']+store_mean['wallet_weixin']+store_mean['alipay_huabei']
store_mean
# trsc = trsc.merge(payw_sum[['store_id','payw_offer']],on='store_id')

,store_id,type,status,effective_amount,original_amount,bankcard_debit,wallet_weixin,wallet_alipay,alipay_huabei,discount_channel,discount_channel_mch,discount_channel_mch_top_up,hongbao_channel,price_reduced,if_price_reduced,payw_offer
0,0030528d-b8f4-ab65-7d89-15b9ee2d3426,1.000000,8.958015,11935.496183,11935.496183,1,1,1,1,1.091603,0.0,0.000000,0.053435,0.000000,0.000000,4
1,0057373d-a954-4e4f-a13a-cf7f091fb1a5,1.000000,0.446154,152355.076923,152355.076923,0,0,1,1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,2
2,005965d9-d80b-40cc-aee0-d53baea6b340,1.000000,0.907692,79264.676923,79264.676923,1,1,1,1,1.261538,0.0,0.000000,0.000000,0.000000,0.000000,4
3,0071e175-90ca-4024-9b54-a2e61326a312,0.996475,6.005287,4452.136412,4452.136412,1,1,1,1,0.685583,0.0,0.000000,0.004230,0.000000,0.000000,4
4,0076c9f4-45cd-429a-8dc9-efb3b40a5e31,1.000000,0.879004,7922.199288,7922.199288,1,1,1,1,1.106762,0.0,0.000000,0.000000,0.000000,0.000000,4
5,0080c517-a15a-4ee1-87c7-855033ec4cb5,1.000000,0.877644,17249.848943,17249.848943,1,1,1,1,0.894260,0.0,0.000000,0.051360,0.000000,0.000000,4
6,00a9058b-3866-4932-8c1b-4e737ae5be5e,1.000000,0.595238,47476.190476,47476.190476,0,1,1,1,4.761905,0.0,0.000000,0.000000,0.000000,0.000000,3
7,00b5be57-77c5-3319-25e8-85a5cb5bdd50,0.996674,10.791795,1523.519281,1523.521744,1,1,1,1,1.623137,0.0,0.000000,0.264507,0.002464,0.000123,4
8,00c618fa-9886-44e3-89b5-56094649893c,1.000000,0.516129,166335.483871,166335.483871,1,1,1,1,2.967742,0.0,0.000000,0.000000,0.000000,0.000000,4
9,00f57c0a-4873-4b95-873a-2ce703688b8a,1.000000,0.956003,2573.780761,2573.780761,1,1,1,1,1.009694,0.0,0.000000,0.117823,0.000000,0.000000,4


In [13]:
trsc.drop(columns=['effective_amount','bankcard_debit','wallet_weixin','wallet_alipay','alipay_huabei'],inplace=True)

## 特征4：（商户提供的）活动

bool(discount_channel & discount_channel &...       )

In [14]:
store_mean['discount']=store_mean['discount_channel'].astype(bool).astype(int)+store_mean['discount_channel_mch'].astype(bool).astype(int)+store_mean['hongbao_channel'].astype(bool).astype(int)+store_mean['discount_channel_mch_top_up'].astype(bool).astype(int)
trsc = trsc.merge(store_mean[['store_id','discount']],on='store_id')

In [15]:
store_mean['if_discount']=store_mean['discount_channel'].astype(bool)+store_mean['discount_channel_mch'].astype(bool)+store_mean['hongbao_channel'].astype(bool)+store_mean['discount_channel_mch_top_up'].astype(bool)
sum(store_mean.if_discount)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


2339

In [16]:
trsc.drop(columns=['discount_channel','discount_channel_mch','discount_channel_mch_top_up','hongbao_channel'],inplace=True)

In [17]:
trsc

,type,status,store_id,merchant_id,original_amount,pt,price_reduced,if_price_reduced,discount
0,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,500,2018-04-14,0,0,2
1,1,0,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,1600,2018-04-14,0,0,2
2,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,1600,2018-04-16,0,0,2
3,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,6400,2018-04-16,0,0,2
4,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,1800,2018-05-05,0,0,2
5,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,800,2018-05-05,0,0,2
6,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,1600,2018-04-09,0,0,2
7,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,1500,2018-05-05,0,0,2
8,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,800,2018-05-04,0,0,2
9,1,1,016abd87-20c2-426e-a282-c98dcc953eb2,e4b5c7eb-f3e7-4990-9c44-402f86da02a4,600,2018-04-16,0,0,2


## 不看throwpoint后的交易数据：

In [18]:
lable = pd.read_csv(LBL,engine='python')
lable.drop(columns=['Unnamed: 0'],inplace=True)

In [19]:
trsc = trsc.merge(lable,on='store_id')

In [20]:
trsc['pt'] = pd.to_datetime(trsc.pt, format='%Y-%m-%d')
trsc['pt_trpt'] = pd.to_datetime(trsc.pt_trpt, format='%Y-%m-%d')

In [21]:
import datetime

trsc['pt_delta'] =  trsc['pt_trpt']-trsc['pt']
trsc = trsc[(trsc[['pt_delta']]>datetime.timedelta(days=0)).all(axis=1)]
trsc.drop(columns=['pt_delta'],inplace=True)

## 特征5： 日均交易笔数、日均交易额、单店日均交易额在该行业占比

#operate_state 

#daily_amount

In [22]:
mch = pd.read_csv(M,engine='python',sep=' ')
# mch

In [23]:
trsc = trsc.merge(mch,on='merchant_id')
trsc['count'] = int(1)

In [24]:
store_sum = trsc.groupby(['store_id']).sum().reset_index()
mch_sum = trsc.groupby(['merchant_id']).sum().reset_index()
store_sum['price_reduced_rate'] = store_sum['if_price_reduced']/store_sum['count']
store_sum['reduce_range'] = store_sum['price_reduced']/store_sum['original_amount']
store_sum['pay_rate'] = store_sum['type']/store_sum['count']
store_sum['success_rate'] = store_sum['status']/store_sum['count']

In [25]:
trsc['days'] = trsc['pt_trpt']-datetime.datetime.strptime('2018-04-01', '%Y-%m-%d')
trsc['days'] = trsc['days'].dt.days
type(trsc.days[1])

numpy.int64

In [33]:
eigen = trsc[['store_id','merchant_id','discount','lost','industry_level1','province','days']]
eigen.drop_duplicates(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
eigen = eigen.merge(store_sum[['store_id','original_amount','count','price_reduced_rate','reduce_range','pay_rate','success_rate']],on='store_id')
eigen = eigen.merge(mch_sum[['merchant_id','original_amount']],on='merchant_id',suffixes=('','_mch'))

In [35]:
eigen['operate_state'] = eigen['original_amount']/eigen['original_amount_mch']
eigen['daily_amount'] = eigen['original_amount']/eigen['days']
eigen['daily_count'] = eigen['count']/trsc['days']

## 特征1：单笔均额

mean(original_amount)

In [36]:
eigen.drop(columns=['merchant_id','days','original_amount','count','original_amount_mch'],inplace=True)
eigen = eigen.merge(store_mean[['store_id','original_amount','payw_offer']],on='store_id',suffixes=('','_mean'))

In [37]:
eigen.to_csv(EIGEN)